In [ ]:
# ===============================
# 🚀 STEP 0: SETUP
# ===============================

# Install Kaggle CLI if not installed
!pip install -q kaggle

# Upload kaggle.json to authenticate Kaggle API
from google.colab import files
files.upload()

# Move kaggle.json to correct directory
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# ===============================
# 🚀 STEP 1: DOWNLOAD & UNZIP DATASET
# ===============================

# Download Stanford Cars dataset (by classes folder)
!kaggle datasets download -d jutrera/stanford-car-dataset-by-classes-folder

# Unzip
!unzip -q stanford-car-dataset-by-classes-folder.zip

# Check contents
!ls

# ===============================
# 🚀 STEP 2: IMPORT LIBRARIES
# ===============================

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np

# ===============================
# 🚀 STEP 3: MOUNT GOOGLE DRIVE
# ===============================

from google.colab import drive
drive.mount('/content/drive')

# ===============================
# 🚀 STEP 4: DATA GENERATORS
# ===============================

train_dir = '/content/car_data/car_data/train'
test_dir = '/content/car_data/car_data/test'

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# ===============================
# 🚀 STEP 5: BUILD MODEL (RESNET50)
# ===============================

base_model = ResNet50(input_shape=IMG_SIZE + (3,), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base model initially

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# ===============================
# 🚀 STEP 6: TRAIN MODEL (FROZEN BASE)
# ===============================

# Define callbacks to save model to Google Drive
checkpoint_path_frozen = '/content/drive/MyDrive/stanford_car_resnet50_model_frozen.h5'

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path_frozen,
                                                monitor='val_loss',
                                                save_best_only=True)

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[early_stop, checkpoint]
)

# ===============================
# 🚀 STEP 7: FINE-TUNE MODEL
# ===============================

# Unfreeze base model for fine-tuning
base_model.trainable = True

# Compile with lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define fine-tuning checkpoint path on Drive
checkpoint_path_finetuned = '/content/drive/MyDrive/stanford_car_resnet50_model_finetuned.h5'

checkpoint_ft = tf.keras.callbacks.ModelCheckpoint(checkpoint_path_finetuned,
                                                   monitor='val_loss',
                                                   save_best_only=True)

history_ft = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[early_stop, checkpoint_ft]
)

# ===============================
# 🚀 STEP 8: PLOT TRAINING CURVES
# ===============================

plt.plot(history.history['accuracy'], label='Train Acc (Frozen)')
plt.plot(history.history['val_accuracy'], label='Val Acc (Frozen)')
plt.plot(history_ft.history['accuracy'], label='Train Acc (Finetuned)')
plt.plot(history_ft.history['val_accuracy'], label='Val Acc (Finetuned)')
plt.legend()
plt.show()

# ===============================
# 🚀 STEP 9: EVALUATE FINAL MODEL
# ===============================

test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy after fine-tuning: {test_acc*100:.2f}%")

# ===============================
# 🚀 STEP 10: SAVE FINAL MODEL TO DRIVE (OPTIONAL)
# ===============================

final_model_path = '/content/drive/MyDrive/stanford_car_resnet50_model_final.h5'
model.save(final_model_path)

print(f"✅ Final model saved to: {final_model_path}")


In [ ]:
# ===============================
# 🚀 FIX: ENSURE DATASET FOLDERS EXIST
# ===============================

import os

# Check if folder exists, if not, download and unzip
if not os.path.exists('/content/car_data/car_data/train'):
    print("🔄 Dataset not found. Downloading and unzipping...")

    # Install Kaggle if needed
    !pip install -q kaggle

    # Authenticate Kaggle
    from google.colab import files
    files.upload()  # Upload kaggle.json again if needed

    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json

    # Download dataset
    !kaggle datasets download -d jutrera/stanford-car-dataset-by-classes-folder

    # Unzip dataset
    !unzip -q stanford-car-dataset-by-classes-folder.zip

    print("✅ Dataset ready.")
else:
    print("✅ Dataset folder already exists.")


In [ ]:
# ===============================
# 🚀 CONTINUATION TRAINING CODE
# ===============================

# 🔷 Install Kaggle (if not already installed)
!pip install -q kaggle

# 🔷 Import libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

# 🔷 Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# ===============================
# 🚀 RELOAD DATASET
# ===============================

# Download dataset if not already in session
!kaggle datasets download -d jutrera/stanford-car-dataset-by-classes-folder
!unzip -q stanford-car-dataset-by-classes-folder.zip

# ===============================
# 🚀 DATA GENERATORS
# ===============================

train_dir = '/content/car_data/car_data/train'
test_dir = '/content/car_data/car_data/test'

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# ===============================
# 🚀 LOAD SAVED FINETUNED MODEL
# ===============================

model_path = '/content/drive/MyDrive/stanford_car_resnet50_model_finetuned.h5'  # adjust if your path differs
model = load_model(model_path)
model.summary()

# ===============================
# 🚀 PREPARE FOR CONTINUED TRAINING
# ===============================

# Ensure base model is trainable
base_model = model.layers[0]
base_model.trainable = True

# Compile with low learning rate for continued fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# ===============================
# 🚀 CONTINUE TRAINING
# ===============================

checkpoint_path_continue = '/content/drive/MyDrive/stanford_car_resnet50_model_finetuned_continued.h5'

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path_continue,
                                                monitor='val_loss',
                                                save_best_only=True)

history_continue = model.fit(
    train_generator,
    epochs=10,  # adjust as needed
    validation_data=val_generator,
    callbacks=[early_stop, checkpoint]
)

# ===============================
# 🚀 PLOT CONTINUED TRAINING CURVES
# ===============================

plt.plot(history_continue.history['accuracy'], label='Train Acc (Continued)')
plt.plot(history_continue.history['val_accuracy'], label='Val Acc (Continued)')
plt.title('Continued Fine-Tuning Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# ===============================
# 🚀 FINAL EVALUATION
# ===============================

test_loss, test_acc = model.evaluate(test_generator)
print(f"✅ Test Accuracy after continued fine-tuning: {test_acc*100:.2f}%")


In [ ]:
# ===============================
# 🚀 CONTINUE TRAINING FOR 20 MORE EPOCHS (FULL SAFE VERSION)
# ===============================

# 🔷 Install Kaggle if needed
!pip install -q kaggle

# 🔷 Import libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import os

# ===============================
# 🚀 MOUNT GOOGLE DRIVE
# ===============================

from google.colab import drive
drive.mount('/content/drive')

# ===============================
# 🚀 ENSURE DATASET IS AVAILABLE
# ===============================

# Check if dataset exists, if not, download and unzip
if not os.path.exists('/content/car_data/car_data/train'):
    print("🔄 Dataset not found. Downloading and unzipping...")

    # Upload kaggle.json if needed
    from google.colab import files
    files.upload()

    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json

    # Download dataset
    !kaggle datasets download -d jutrera/stanford-car-dataset-by-classes-folder
    !unzip -q stanford-car-dataset-by-classes-folder.zip

    print("✅ Dataset ready.")
else:
    print("✅ Dataset already exists.")

# ===============================
# 🚀 DATA GENERATORS
# ===============================

train_dir = '/content/car_data/car_data/train'
test_dir = '/content/car_data/car_data/test'

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# ===============================
# 🚀 LOAD YOUR LAST SAVED MODEL
# ===============================

model_path = '/content/drive/MyDrive/stanford_car_resnet50_model_finetuned_continued.h5'
model = load_model(model_path)
model.summary()

# ===============================
# 🚀 ENSURE BASE MODEL IS TRAINABLE
# ===============================

base_model = model.layers[0]
base_model.trainable = True

# Re-compile with low learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# ===============================
# 🚀 DEFINE CHECKPOINT & EARLY STOPPING
# ===============================

checkpoint_path_continue_20 = '/content/drive/MyDrive/stanford_car_resnet50_model_finetuned_continued_20more.h5'

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path_continue_20,
                                                monitor='val_loss',
                                                save_best_only=True)

# ===============================
# 🚀 TRAIN FOR 20 MORE EPOCHS
# ===============================

history_continue = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[early_stop, checkpoint]
)

# ===============================
# 🚀 PLOT TRAINING CURVES
# ===============================

plt.plot(history_continue.history['accuracy'], label='Train Acc (Continued 20)')
plt.plot(history_continue.history['val_accuracy'], label='Val Acc (Continued 20)')
plt.title('Continued Fine-Tuning Accuracy (20 more epochs)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# ===============================
# 🚀 FINAL EVALUATION
# ===============================

test_loss, test_acc = model.evaluate(test_generator)
print(f"✅ Test Accuracy after additional 20 epochs: {test_acc*100:.2f}%")


In [ ]:
# ================================
# 🔧 Stanford Cars MMC Inference +
# 🔢 Manual Accuracy Evaluation
# ================================

# 1. Install requirements
!pip install -q tensorflow

# 2. Import libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from google.colab import drive, files
from PIL import Image
import os

# 3. Mount Google Drive
drive.mount('/content/drive')

# 4. Load your trained model
model_path = '/content/drive/MyDrive/stanford_car_resnet50_model_finetuned_continued_20more.h5'
model = tf.keras.models.load_model(model_path)
print(f"✅ Model loaded from: {model_path}")

# 5. Load class labels (based on your training data)
# Update this if your dataset structure is different
train_dir = '/content/car_data/car_data/train'  # Change if needed

datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
generator = datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=1, class_mode='categorical')
class_indices = generator.class_indices
class_labels = list(class_indices.keys())
print("✅ Class labels loaded.")

# 6. Upload multiple images
uploaded = files.upload()
image_filenames = list(uploaded.keys())

# 7. Predict each image and optionally collect accuracy
correct = 0
total = 0

# 🔁 Optional: Map your uploaded image filenames to their correct class (ground truth)
# Example: {'car1.jpg': 'Aston Martin DB9 Coupe 2012', ...}
ground_truth = {
    'img1.jpg': 'Aston Martin DB9 Coupe 2012',
    'img2.jpg': 'Honda Accord Sedan 2012',
    'img3.jpg': 'Tesla Model S 2012'
    # 👆 Replace with your actual uploaded image filenames and true labels
}

for filename in image_filenames:
    # Load and preprocess image
    img = image.load_img(filename, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Predict
    preds = model.predict(img_array)
    pred_class_index = np.argmax(preds)
    confidence = np.max(preds) * 100
    predicted_label = class_labels[pred_class_index]

    # Print result
    print(f"\n🖼️ Image: {filename}")
    print(f"🔍 Predicted: {predicted_label} ({confidence:.2f}%)")

    # Display image
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Predicted: {predicted_label}\nConfidence: {confidence:.2f}%")
    plt.show()

    # Check against ground truth (if provided)
    if filename in ground_truth:
        actual_label = ground_truth[filename]
        total += 1
        if predicted_label == actual_label:
            correct += 1
        else:
            print(f"❌ Wrong! Actual: {actual_label}")

# 8. Print final accuracy
if total > 0:
    acc = (correct / total) * 100
    print(f"\n✅ Accuracy on uploaded images: {acc:.2f}% ({correct}/{total})")
else:
    print("\n⚠️ No ground truth provided. Accuracy not calculated.")


In [ ]:
# ===============================
# 🚀 IMPORT LIBRARIES
# ===============================
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import matplotlib.pyplot as plt
import os
from google.colab import files
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ===============================
# 🚀 LOAD MODEL
# ===============================
model_path = '/content/drive/MyDrive/stanford_car_resnet50_model_finetuned_continued_20more.h5'
model = load_model(model_path)
model.summary()

# ===============================
# 🚀 LOAD LABELS (CLASS INDICES)
# ===============================
# Assuming your test set structure is still present
test_dir = '/content/car_data/car_data/test'

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

class_indices = test_generator.class_indices
idx_to_class = {v: k for k, v in class_indices.items()}

# ===============================
# 🚀 EVALUATE ACCURACY
# ===============================
loss, accuracy = model.evaluate(test_generator)
print(f"✅ Test Accuracy: {accuracy*100:.2f}%")

# ===============================
# 🚀 UPLOAD IMAGES FOR PREDICTION
# ===============================
uploaded = files.upload()

for fname in uploaded.keys():
    # Load and preprocess
    img_path = fname
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    predictions = model.predict(img_array)
    predicted_class = idx_to_class[np.argmax(predictions)]

    # Plot
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"🚗 Predicted: {predicted_class}")
    plt.show()


In [ ]:
# ===============================
# 🔹 INSTALL & IMPORTS
# ===============================
!pip install -q kaggle

import os
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ===============================
# 🔹 MOUNT GOOGLE DRIVE
# ===============================
from google.colab import drive
drive.mount('/content/drive')

# ===============================
# 🔹 DOWNLOAD & EXTRACT DATASET
# ===============================
if not os.path.exists("/content/car_data/car_data/train"):
    print("🔄 Downloading and extracting Stanford Cars dataset...")

    uploaded = files.upload()  # Upload your kaggle.json
    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json

    !kaggle datasets download -d jutrera/stanford-car-dataset-by-classes-folder
    !unzip -q stanford-car-dataset-by-classes-folder.zip -d /content/car_data

    print("✅ Dataset downloaded and extracted to /content/car_data")
else:
    print("✅ Dataset already exists at /content/car_data")

# ===============================
# 🔹 LOAD CLASS LABELS
# ===============================
IMG_SIZE = (224, 224)

temp_datagen = ImageDataGenerator()
temp_generator = temp_datagen.flow_from_directory(
    '/content/car_data/car_data/train',
    target_size=IMG_SIZE,
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)

class_indices = temp_generator.class_indices
class_labels = list(class_indices.keys())

print(f"✅ Loaded {len(class_labels)} class labels.")

# ===============================
# 🔹 LOAD MODEL FROM DRIVE
# ===============================
model_path = '/content/drive/MyDrive/stanford_car_resnet50_model_finetuned_continued_20more.h5'

model = load_model(model_path)
print("✅ Model loaded successfully.")
model.summary()

# ===============================
# 🔹 UPLOAD IMAGE(S) FOR PREDICTION
# ===============================
uploaded = files.upload()

for fname in uploaded.keys():
    # Load and preprocess image
    img_path = os.path.join("/content", fname)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Predict
    predictions = model.predict(img_array)
    predicted_index = np.argmax(predictions)
    predicted_class = class_labels[predicted_index]
    confidence = np.max(predictions)

    # Display
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Predicted: {predicted_class} ({confidence * 100:.2f}%)")
    plt.show()


In [ ]:
from google.colab import files
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import preprocess_input

uploaded = files.upload()

for fname in uploaded.keys():
    img_path = os.path.join("/content", fname)
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = preprocess_input(np.expand_dims(img_array, axis=0))

    preds = model.predict(img_array)
    top_index = np.argmax(preds)
    predicted_class = class_names[top_index]
    confidence = preds[0][top_index]

    # Show result
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Prediction: {predicted_class} ({confidence*100:.2f}%)")
    plt.show()


In [ ]:
# ===============================
# 🔹 INSTALL DEPENDENCIES
# ===============================
!pip install -q tensorflow matplotlib

import os
import numpy as np
import scipy.io
import shutil
import matplotlib.pyplot as plt
from google.colab import drive, files
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

# ===============================
# 🔹 MOUNT GOOGLE DRIVE
# ===============================
drive.mount('/content/drive')

# ===============================
# 🔹 DOWNLOAD & EXTRACT DATASET
# ===============================
import urllib.request
import tarfile

dataset_dir = "/content/stanford_cars"
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)

if not os.path.exists(os.path.join(dataset_dir, "cars_train")):
    print("🔽 Downloading dataset...")
    urllib.request.urlretrieve(
        "http://ai.stanford.edu/~jkrause/car196/cars_train.tgz",
        os.path.join(dataset_dir, "cars_train.tgz")
    )
    urllib.request.urlretrieve(
        "http://ai.stanford.edu/~jkrause/car196/cars_test.tgz",
        os.path.join(dataset_dir, "cars_test.tgz")
    )
    urllib.request.urlretrieve(
        "http://ai.stanford.edu/~jkrause/car196/cars_train_annos.mat",
        os.path.join(dataset_dir, "cars_train_annos.mat")
    )
    urllib.request.urlretrieve(
        "http://ai.stanford.edu/~jkrause/car196/cars_test_annos_withlabels.mat",
        os.path.join(dataset_dir, "cars_test_annos_withlabels.mat")
    )
    urllib.request.urlretrieve(
        "http://ai.stanford.edu/~jkrause/car196/cars_meta.mat",
        os.path.join(dataset_dir, "cars_meta.mat")
    )

    print("📦 Extracting...")
    with tarfile.open(os.path.join(dataset_dir, "cars_train.tgz")) as tar:
        tar.extractall(path=dataset_dir)
    with tarfile.open(os.path.join(dataset_dir, "cars_test.tgz")) as tar:
        tar.extractall(path=dataset_dir)

    print("✅ Downloaded and extracted Stanford Cars dataset")
else:
    print("✅ Dataset already downloaded.")

# ===============================
# 🔹 LOAD CLASS LABELS
# ===============================
meta = scipy.io.loadmat(os.path.join(dataset_dir, "cars_meta.mat"))
class_names = [x[0] for x in meta["class_names"][0]]
print(f"✅ Loaded {len(class_names)} classes.")

# ===============================
# 🔹 LOAD YOUR TRAINED MODEL
# ===============================
model_path = '/content/drive/MyDrive/stanford_car_resnet50_model_finetuned_continued_20more.h5'
model = load_model(model_path)
print("✅ Model loaded.")

# ===============================
# 🔹 UPLOAD & PREDICT
# ===============================
uploaded = files.upload()

for fname in uploaded.keys():
    img_path = os.path.join("/content", fname)
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = preprocess_input(np.expand_dims(img_array, axis=0))

    preds = model.predict(img_array)
    top_index = np.argmax(preds)
    predicted_class = class_names[top_index]
    confidence = preds[0][top_index]

    # Show result
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Prediction: {predicted_class} ({confidence*100:.2f}%)")
    plt.show()


In [ ]:
# === Step 1: Setup Kaggle & Download Dataset ===
!pip install -q kaggle

from google.colab import files
files.upload()  # Upload your kaggle.json here

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download Stanford Cars dataset
!kaggle datasets download -d jutrera/stanford-car-dataset-by-classes-folder

# Unzip dataset
!unzip -q stanford-car-dataset-by-classes-folder.zip

# === Step 2: Extract class names from train folder ===
import os

train_dir = '/content/car_data/car_data/train'
class_names = sorted(os.listdir(train_dir))
print(f"✅ Found {len(class_names)} classes.")
print(class_names[:10])  # show first 10 classes as example

# === Step 3: Load libraries & your model ===
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
import numpy as np
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

model_path = '/content/drive/MyDrive/stanford_car_resnet50_model_finetuned_continued_20more.h5'
model = tf.keras.models.load_model(model_path)
print("✅ Model loaded.")

IMG_SIZE = (224, 224)

# === Step 4: Upload image(s) for prediction ===
print("Upload image(s) to predict the make and model:")
uploaded = files.upload()

for img_name in uploaded.keys():
    print(f"\nProcessing {img_name}...")

    # Load and preprocess image
    img = image.load_img(img_name, target_size=IMG_SIZE)
    img_array = image.img_to_array(img)
    img_array_exp = np.expand_dims(img_array, axis=0)
    img_preprocessed = preprocess_input(img_array_exp)

    # Predict
    preds = model.predict(img_preprocessed)
    pred_index = np.argmax(preds)
    pred_class = class_names[pred_index]
    confidence = preds[0][pred_index]

    # Print prediction and confidence
    print(f"Predicted Make & Model: {pred_class}")
    print(f"Confidence: {confidence * 100:.2f}%")

    # Display image with prediction
    plt.imshow(img)
    plt.title(f"Prediction: {pred_class}\nConfidence: {confidence*100:.2f}%")
    plt.axis('off')
    plt.show()


In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import preprocess_input
import tensorflow as tf
from google.colab import files

# Upload multiple images
uploaded_files = files.upload()

for image_path in uploaded_files.keys():
    print(f"\nProcessing: {image_path}")

    # Load image and preprocess
    img = Image.open(image_path).convert('RGB').resize((224, 224))
    img_array = np.array(img)
    img_preprocessed = preprocess_input(np.expand_dims(img_array, axis=0))  # Add batch dim

    # Predict
    preds = model.predict(img_preprocessed)

    # Get top 3 predictions
    top_k = 3
    top_indices = preds[0].argsort()[-top_k:][::-1]

    print("Top 3 predictions:")
    for i in top_indices:
        print(f"{class_names[i]}: {preds[0][i]*100:.2f}%")

    # Optional: display image with highest prediction label
    plt.imshow(img)
    plt.title(f"Top prediction: {class_names[top_indices[0]]}")
    plt.axis('off')
    plt.show()


In [ ]:
# --- convert_h5_to_pt_drive.py ---
import torch
from torchvision import models
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from google.colab import drive

# --- Step 0: Mount Google Drive ---
drive.mount('/content/drive')

# --- Step 1: Load Keras model from Drive ---
keras_model_path = "/content/drive/MyDrive/stanford_car_resnet50_model_finetuned_continued_20more.h5"
keras_model = load_model(keras_model_path)
print("✅ Keras model loaded from Drive.")

# --- Step 2: Rebuild same ResNet50 architecture in PyTorch ---
num_classes = keras_model.output_shape[1]  # should be 196 for Cars196
pt_model = models.resnet50(pretrained=False)
pt_model.fc = torch.nn.Linear(pt_model.fc.in_features, num_classes)

# --- Step 3: Copy weights from Keras to PyTorch ---
# Function to copy weights layer by layer
keras_layers = [layer for layer in keras_model.layers if len(layer.get_weights()) > 0]

for layer in keras_layers:
    weights = layer.get_weights()
    if 'conv' in layer.name and len(weights) > 0:
        w = np.transpose(weights[0], (3, 2, 0, 1))
        pt_layer = dict(pt_model.named_modules()).get(layer.name)
        if pt_layer is not None:
            pt_layer.weight.data = torch.from_numpy(w).float()
        if len(weights) > 1:
            pt_layer.bias.data = torch.from_numpy(weights[1]).float()
    elif 'bn' in layer.name and len(weights) > 0:
        pt_layer = dict(pt_model.named_modules()).get(layer.name)
        if pt_layer is not None:
            pt_layer.weight.data = torch.from_numpy(weights[0]).float()
            pt_layer.bias.data = torch.from_numpy(weights[1]).float()
            pt_layer.running_mean = torch.from_numpy(weights[2]).float()
            pt_layer.running_var = torch.from_numpy(weights[3]).float()
    elif 'dense' in layer.name or 'fc' in layer.name:
        w, b = weights
        pt_model.fc.weight.data = torch.from_numpy(w.T).float()
        pt_model.fc.bias.data = torch.from_numpy(b).float()

print("✅ Weights copied to PyTorch model.")

# --- Step 4: Save PyTorch model back to Drive ---
pt_model_path = "/content/drive/MyDrive/stanford_car_resnet50_model_finetuned.pt"
torch.save(pt_model, pt_model_path)
print(f"✅ PyTorch model saved to Drive at {pt_model_path}")


In [ ]:
# ============================================
# 🚀 Convert Keras .h5 to TFLite and SAVE TO DRIVE
# ============================================
from google.colab import drive
import os, urllib.request, scipy.io
import tensorflow as tf

# ---- 1) Mount Google Drive ----
drive.mount('/content/drive', force_remount=True)

# ---- 2) Configure your paths (EDIT if your filenames/folders differ) ----
MODEL_H5 = '/content/drive/MyDrive/stanford_car_resnet50_model_finetuned_continued_20more.h5'
DRIVE_OUT_DIR = '/content/drive/MyDrive/exports_for_pi'   # output folder in Drive
CARS_META_URL = 'http://ai.stanford.edu/~jkrause/car196/cars_meta.mat'

# Create output directory in Drive
os.makedirs(DRIVE_OUT_DIR, exist_ok=True)

# ---- 3) Verify model exists on Drive ----
if not os.path.exists(MODEL_H5):
    raise FileNotFoundError(f"❌ Model .h5 not found at: {MODEL_H5}\n"
                            f"➡️  Update MODEL_H5 to the correct path in your Drive.")

print(f"✅ Found model: {MODEL_H5}")

# ---- 4) Ensure we have cars_meta.mat (for labels) ----
meta_local_path = '/content/cars_meta.mat'
if not os.path.exists(meta_local_path):
    print("📥 Downloading cars_meta.mat for class labels...")
    urllib.request.urlretrieve(CARS_META_URL, meta_local_path)
else:
    print("✅ cars_meta.mat already present locally")

# ---- 5) Build labels.txt directly in Drive ----
meta = scipy.io.loadmat(meta_local_path)
class_names = [x[0] for x in meta["class_names"][0]]

labels_drive_path = os.path.join(DRIVE_OUT_DIR, 'labels.txt')
with open(labels_drive_path, 'w') as f:
    f.writelines(name + '\n' for name in class_names)

print(f"✅ Saved labels.txt to Drive: {labels_drive_path}")

# ---- 6) Load .h5 and convert to TFLite (float model; optimized) ----
print("📦 Loading Keras model...")
model = tf.keras.models.load_model(MODEL_H5)

print("🔄 Converting to TensorFlow Lite...")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# Keeps float precision; applies graph/transformation optimizations
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# ---- 7) Save TFLite directly to Drive ----
tflite_drive_path = os.path.join(DRIVE_OUT_DIR, 'cars_model.tflite')
with open(tflite_drive_path, 'wb') as f:
    f.write(tflite_model)

print(f"✅ Saved TFLite model to Drive: {tflite_drive_path}")

print("\n🎯 DONE! Files in Drive:")
print(f"   • {tflite_drive_path}")
print(f"   • {labels_drive_path}")


In [ ]:
# === Setup Kaggle & Download Stanford Car Dataset ===

import os
import json
from google.colab import files

# 1. Install Kaggle API
!pip install -q kaggle

# 2. Upload your kaggle.json (API key)
uploaded = files.upload()  # Select kaggle.json

# 3. Move it to correct place
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 4. Download the Stanford Cars dataset (by classes folder)
DATASET = "jutrera/stanford-car-dataset-by-classes-folder"
!kaggle datasets download -d {DATASET} -p /content/

# 5. Unzip the downloaded file
!unzip -q /content/stanford-car-dataset-by-classes-folder.zip -d /content/car_data

# 6. Define path to training folder (adjust if folder structure is different)
train_dir = "/content/car_data/car_data/train"

# 7. Read class names
class_names = sorted(os.listdir(train_dir))
print(f"✅ Found {len(class_names)} classes.")
print("First 10 classes:", class_names[:10])

# 8. Save class names in JSON
with open("car_classes.json", "w") as f:
    json.dump(class_names, f)

print("✅ car_classes.json saved")

# 9. Download the JSON file to your local machine
files.download("car_classes.json")

In [ ]:
import os
import json
from google.colab import files

# After unzipping, look through car_data directory for the right train folder
base_dir = "/content/car_data"

# Walk through folders and find one that looks like "train" or "cars_train"
train_dir = None
for root, dirs, files_in_dir in os.walk(base_dir):
    for d in dirs:
        if "train" in d.lower():  # catches "train", "cars_train", etc.
            train_dir = os.path.join(root, d)
            break
    if train_dir:
        break

if train_dir is None:
    raise FileNotFoundError("❌ Could not find a training folder. Please check dataset structure.")

print(f"✅ Training directory found: {train_dir}")

# Get class names
class_names = sorted(os.listdir(train_dir))
print(f"✅ Found {len(class_names)} classes.")
print("First 10 classes:", class_names[:10])

# Save class names into JSON
with open("car_classes.json", "w") as f:
    json.dump(class_names, f)

print("✅ car_classes.json saved")

# Download JSON to your computer
files.download("car_classes.json")


In [ ]:
import os
from google.colab import files

# Base unzip directory
base_dir = "/content/car_data"

# 🔍 Auto-detect training folder
train_dir = None
for root, dirs, files_in_dir in os.walk(base_dir):
    for d in dirs:
        if "train" in d.lower():  # catches "train", "cars_train", etc.
            train_dir = os.path.join(root, d)
            break
    if train_dir:
        break

if train_dir is None:
    raise FileNotFoundError("❌ Could not find a training folder. Please check dataset structure.")

print(f"✅ Training directory found: {train_dir}")

# 📂 Read class names
class_names = sorted(os.listdir(train_dir))
print(f"✅ Found {len(class_names)} classes.")
print("First 10 classes:", class_names[:10])

# 📝 Save to TXT (one class per line)
with open("car_classes.txt", "w") as f:
    f.write("\n".join(class_names))

print("✅ car_classes.txt saved")

# 💾 Download to your computer
files.download("car_classes.txt")
